# Data Splits

We want to split the SynFerm data set into train, validation and test data.
For all splits, we will do 9 random repetitions.
For 1D and 2D split, which both use 3 different groups to split on, these will divide into 3 random repetitions for each of the 3 groups. 

### 0D Split
For the 0D split, we use a random train-test split.
We use a 80/10/10 split into train, val, and test set.

### 1D Split
For the 1D split, we use a (1D) GroupShuffleSplit.
Each individual split will be 70/15/15 train/test (of groups not samples!).
As groups, we use either initiator, monomer, or terminator.

### 2D Split
For the 2D split, we use a (2D) GroupShuffleSplit.
Each individual split will use 20% of groups as test set and 25% of remaining groups as validation set. 
Due to the dimensionality, this means we expect 0.2 * 0.2 = 4% of samples in the test and validation set and 0.800^2 * 0.75^2 = 36.0% of samples in the training set.
The remaining samples are not used to prevent leakage.
As groups, we use either \[initiator, monomer], \[monomer, terminator] or \[initiator, terminator].

### 3D Split
For the 3D split, we use a (3D) GroupShuffleSplit.
Each individual split will use 25% of groups as test set, 33% of remaining groups as validation set, and the remaining groups as training set.
Due to the dimensionality, this means we expect 0.25^3 = 1.5% of samples in the test and validation set and 0.75^3 * 0.67^3 = 12.5% of sample in the training set.

In [1]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit

from src.definitions import DATA_DIR
from src.util.train_test_split import GroupShuffleSplitND
from util import write_indices_and_stats

In [2]:
# Load data
data_filename = "synferm_dataset_2023-12-20_39486records.csv"
data_name = data_filename.rsplit("_", maxsplit=1)[0]
df = pd.read_csv(DATA_DIR / "curated_data" / data_filename)
df.shape

(39486, 27)

In [3]:
df.head()

,I_long,M_long,T_long,product_A_smiles,I_smiles,M_smiles,T_smiles,reaction_smiles,reaction_smiles_atom_mapped,experiment_id,...,binary_H,scaled_A,scaled_B,scaled_C,scaled_D,scaled_E,scaled_F,scaled_G,scaled_H,major_A-C
0,2-Pyr003,Fused002,TerABT004,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,Nc1ccc(F)cc1S,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:1](=[O:2])[c:15]1[cH:16][cH:18][...,56113,...,0,0.036021,0.003427,0.0,0.020975,0.002958,0.941981,0.914281,0.00000,A
1,2-Pyr003,Fused002,TerABT007,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,Nc1cc(Br)ccc1S,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:1](=[O:2])[c:15]1[cH:16][cH:18][...,56114,...,0,0.000000,0.000000,0.0,0.006159,0.364398,0.928851,1.106548,0.00000,no_product
2,2-Pyr003,Fused002,TerABT013,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,Nc1cc(C(F)(F)F)ccc1S,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:1](=[O:2])[c:15]1[cH:16][cH:18][...,56106,...,1,0.000000,0.000000,0.0,0.014212,2.166420,1.013596,0.537785,0.05686,no_product
3,2-Pyr003,Fused002,TerABT014,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,Nc1ccc(Cl)cc1S,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:1](=[O:2])[c:15]1[cH:16][cH:18][...,56112,...,0,0.028915,0.005039,0.0,0.015578,0.504057,0.992614,0.890646,0.00000,A
4,2-Pyr003,Fused002,TerTH001,COc1ccc(CCOC(=O)N2C[C@H](NC(=O)c3cccc(Cl)n3)[C...,O=C(c1cccc(Cl)n1)[B-](F)(F)F.[K+],COc1ccc(CCOC(=O)N2C[C@@H]3NO[C@]4(OC5(CCCCC5)O...,[Cl-].[NH3+]NC(=S)c1ccccc1,O=C(c1cccc(Cl)n1)[B-](F)(F)F.COc1ccc(CCOC(=O)N...,F[B-](F)(F)[C:1](=[O:2])[c:11]1[cH:12][cH:14][...,56109,...,0,0.350061,0.643219,0.0,0.031689,0.613596,0.109309,0.439018,0.00000,B


In [4]:
# M_long_dia will be to sort diastereomers into the same group on group shuffle splits
diastereomers = {
    "Mon001": "Mon087",
    "Mon003": "Mon078",
    "Mon011": "Mon088",
    "Mon013": "Mon074",
    "Mon014": "Mon090",
    "Mon015": "Mon076",
    "Mon016": "Mon096",
    "Mon017": "Mon075",
    "Mon019": "Mon091",
    "Mon020": "Mon077",
    "Mon080": "Mon010",
}
df["M_long_dia"] = df["M_long"].replace(diastereomers)

## 0D split (not needed, see 0D_80 in truncated splits)

## 0D split final-retrain
To retrain the best model after selection, for the 0D split we only use one fold, split into training and validation set (used for early stopping of FFN training)

In [5]:
splitter = ShuffleSplit(n_splits=1, test_size=0.1, random_state=42)

indices = []
sizes = []
pos_class = []
for idx_train, idx_val in splitter.split(df):
    idx_test = []  # placeholder so we can use the write_indices_and_stats function, just delete fold0_test.csv later
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(), 

        )
    )
    
print(sizes)
print(pos_class)

[(35537, 3949, 0)]
[(array([29166, 20532, 10145]), array([3246, 2272, 1124]), array([0., 0., 0.]))]


In [6]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=0, 
    save_indices=True, 
    train_size="final_retrain"
)

/Users/julian/PycharmProjects/library-generation/notebooks/data_featurization/util.py:91: RuntimeWarning: invalid value encountered in scalar divide
  f"Test samples binary_A has label 1: {count_pos[2][0]} ({count_pos[2][0]/size[2]:.1%})\n"
/Users/julian/PycharmProjects/library-generation/notebooks/data_featurization/util.py:94: RuntimeWarning: invalid value encountered in scalar divide
  f"Test samples binary_B has label 1: {count_pos[2][1]} ({count_pos[2][1]/size[2]:.1%})\n"
/Users/julian/PycharmProjects/library-generation/notebooks/data_featurization/util.py:97: RuntimeWarning: invalid value encountered in scalar divide
  f"Test samples binary_C has label 1: {count_pos[2][2]} ({count_pos[2][2]/size[2]:.1%})\n"
/Users/julian/PycharmProjects/library-generation/notebooks/data_featurization/util.py:104: RuntimeWarning: invalid value encountered in scalar divide
  f"Chance level average precision macro on test set: {np.sum(count_pos[2]) / (3 * size[2]):.3f}\n"
/Users/julian/PycharmProjec

## 1D split

In [7]:
splitter = GroupShuffleSplit(n_splits=3, test_size=0.15, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.15/0.85, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [8]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

# note: for M_long, we need to obtain diastereomer relationships to sort them into the same group
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["M_long_dia"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["M_long_dia"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
        
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["T_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["T_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
print(sizes)
print(pos_class)

[(27196, 5672, 6618), (25001, 8543, 5942), (28990, 5536, 4960), (25495, 9037, 4954), (27626, 6433, 5427), (26904, 6701, 5881), (28292, 5161, 6033), (27420, 5605, 6461), (28020, 5449, 6017)]
[(array([22402, 15429,  7746]), array([4277, 3422, 1582]), array([5733, 3953, 1941])), (array([20347, 14227,  6881]), array([7424, 5302, 2781]), array([4641, 3275, 1607])), (array([24239, 16853,  8631]), array([4214, 3305, 1456]), array([3959, 2646, 1182])), (array([20427, 14342,  6415]), array([8317, 5985, 3464]), array([3668, 2477, 1390])), (array([22505, 15898,  7994]), array([5115, 3473, 1511]), array([4792, 3433, 1764])), (array([22187, 15431,  7698]), array([5309, 3951, 1930]), array([4916, 3422, 1641])), (array([22839, 16777,  8612]), array([4477, 2222, 1088]), array([5096, 3805, 1569])), (array([23560, 18246,  9601]), array([3992, 1341,  324]), array([4860, 3217, 1344])), (array([22754, 16454,  7849]), array([4982, 3519, 2172]), array([4676, 2831, 1248]))]


In [9]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    train_size=""
)

## 1D split for Euan
Requirement: same split logic as above, but with 3x9 folds instead of 3x3 folds

In [10]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.15, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.15/0.85, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [11]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

# note: for M_long, we need to obtain diastereomer relationships to sort them into the same group
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["M_long_dia"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["M_long_dia"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
        
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["T_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["T_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
print(sizes)
print(pos_class)

[(27196, 5672, 6618), (25001, 8543, 5942), (28990, 5536, 4960), (27114, 5545, 6827), (28324, 5141, 6021), (29317, 5570, 4599), (26843, 5898, 6745), (28243, 4478, 6765), (26255, 5471, 7760), (24905, 6069, 8512), (25390, 6389, 7707), (26374, 5859, 7253), (27253, 6278, 5955), (28283, 5914, 5289), (27633, 6778, 5075), (26086, 6882, 6518), (26934, 5804, 6748), (26338, 6517, 6631), (27288, 6022, 6176), (28041, 5283, 6162), (28188, 5644, 5654), (27521, 5507, 6458), (26962, 5899, 6625), (27173, 5259, 7054), (26677, 6279, 6530), (27915, 6268, 5303), (27129, 5337, 7020)]
[(array([22402, 15429,  7746]), array([4277, 3422, 1582]), array([5733, 3953, 1941])), (array([20347, 14227,  6881]), array([7424, 5302, 2781]), array([4641, 3275, 1607])), (array([24239, 16853,  8631]), array([4214, 3305, 1456]), array([3959, 2646, 1182])), (array([22310, 15968,  7898]), array([4258, 2850, 1397]), array([5844, 3986, 1974])), (array([23199, 16441,  8035]), array([4215, 2833, 1498]), array([4998, 3530, 1736])), (

In [12]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    train_size="euan27folds"
)

## 2D split

In [8]:
splitter = GroupShuffleSplitND(n_splits=3, test_size=0.2, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.2/0.8, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [9]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df, groups=df[["I_long", "M_long_dia"]]):
    train, val = next(inner_splitter.split(df.iloc[idx_train_val], groups=df[["I_long", "M_long_dia"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df[["M_long_dia", "T_long"]]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df[["M_long_dia", "T_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df[["I_long", "T_long"]]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df[["I_long", "T_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
        
print(sizes)
print(pos_class)

[(13206, 1738, 1756), (13087, 1922, 1658), (14890, 1415, 1615), (13375, 1546, 1891), (14278, 1574, 1487), (13462, 1461, 1916), (14138, 1398, 1957), (14268, 1619, 1624), (12864, 1791, 1736)]
[(array([10818,  7819,  3570]), array([1404,  959,  516]), array([1505,  998,  536])), (array([10668,  7626,  3429]), array([1683, 1166,  692]), array([1299,  892,  449])), (array([12164,  8560,  4204]), array([1218,  877,  539]), array([1239,  849,  283])), (array([11046,  7905,  3957]), array([1322, 1163,  391]), array([1371,  740,  459])), (array([12408,  9551,  4882]), array([1058,  800,  308]), array([1174,  594,  289])), (array([11314,  7766,  4157]), array([932, 583, 195]), array([1737, 1420,  719])), (array([11569,  7415,  3662]), array([1182,  825,  462]), array([1527, 1229,  535])), (array([11423,  7291,  3659]), array([1426, 1183,  572]), array([1321, 1089,  502])), (array([9914, 5868, 2664]), array([1573, 1305,  799]), array([1505, 1314,  623]))]


In [10]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    train_size=""
)

## 3D split

In [14]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.25, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState (not true, copyPaste error from before. Anyway, not a problem)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.25/0.75, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [15]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df, groups=df[["I_long", "M_long_dia", "T_long"]]):
    train, val = next(inner_splitter.split(df.iloc[idx_train_val], groups=df[["I_long", "M_long_dia", "T_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
print(sizes)
print(pos_class)

[(4481, 613, 834), (4384, 672, 688), (4506, 580, 714), (3999, 554, 735), (5221, 590, 539), (5498, 554, 502), (4626, 672, 515), (5028, 584, 563), (4225, 638, 736)]
[(array([3608, 2827, 1349]), array([517, 319, 180]), array([697, 491, 201])), (array([3674, 2726, 1073]), array([548, 266, 177]), array([553, 435, 230])), (array([3458, 2533, 1416]), array([509, 348, 149]), array([589, 373, 160])), (array([3470, 2729, 1498]), array([408, 148,  56]), array([581, 475, 213])), (array([4275, 3253, 1292]), array([474, 321, 248]), array([445, 268, 124])), (array([4746, 2687, 1420]), array([486, 302, 116]), array([344, 334, 127])), (array([3870, 3192, 1655]), array([584, 394, 132]), array([359, 184,  66])), (array([4388, 2779, 1668]), array([449, 386, 182]), array([443, 298, 113])), (array([3440, 2135,  934]), array([567, 394, 307]), array([598, 498, 191]))]


In [16]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    train_size=""
)

## Control: Check splits

### 1D split

In [17]:
split_dimension = 1
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9): # only these are split on monomers
    
    # import indices
    train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
    val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
    test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()

    # check mutually exclusive
    assert len(np.intersect1d(train_idx, val_idx)) == 0
    assert len(np.intersect1d(train_idx, test_idx)) == 0
    assert len(np.intersect1d(val_idx, test_idx)) == 0

    # check 1D groups are mutually exclusive
    if fold_idx < 3: # first three are split on initiator
        assert len(np.intersect1d(df["I_long"].iloc[train_idx], df["I_long"].iloc[val_idx])) == 0
        assert len(np.intersect1d(df["I_long"].iloc[train_idx], df["I_long"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["I_long"].iloc[val_idx], df["I_long"].iloc[test_idx])) == 0
    elif fold_idx < 6:  # next three are split on monomer
        assert len(np.intersect1d(df["M_long"].iloc[train_idx], df["M_long"].iloc[val_idx])) == 0
        assert len(np.intersect1d(df["M_long"].iloc[train_idx], df["M_long"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["M_long"].iloc[val_idx], df["M_long"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["M_long_dia"].iloc[train_idx], df["M_long_dia"].iloc[val_idx])) == 0
        assert len(np.intersect1d(df["M_long_dia"].iloc[train_idx], df["M_long_dia"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["M_long_dia"].iloc[val_idx], df["M_long_dia"].iloc[test_idx])) == 0
        
    else:  # last three are split on terminator
        assert len(np.intersect1d(df["T_long"].iloc[train_idx], df["T_long"].iloc[val_idx])) == 0
        assert len(np.intersect1d(df["T_long"].iloc[train_idx], df["T_long"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["T_long"].iloc[val_idx], df["T_long"].iloc[test_idx])) == 0


### 2D split

In [18]:
split_dimension = 2
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9):
    # import indices
    train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
    val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
    test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()

    # check mutually exclusive
    assert len(np.intersect1d(train_idx, val_idx)) == 0
    assert len(np.intersect1d(train_idx, test_idx)) == 0
    assert len(np.intersect1d(val_idx, test_idx)) == 0

    # check 2D groups are mutually exclusive
    if fold_idx < 3: # first three are split on initiator and monomer
        assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia"]].iloc[train_idx]), np.unique(df[["I_long", "M_long_dia"]].iloc[val_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia"]].iloc[train_idx]), np.unique(df[["I_long", "M_long_dia"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia"]].iloc[val_idx]), np.unique(df[["I_long", "M_long_dia"]].iloc[test_idx]))) == 0
    elif fold_idx < 6:  # next three are split on monomer and terminator
        assert len(np.intersect1d(np.unique(df[["M_long", "T_long"]].iloc[train_idx]), np.unique(df[["M_long", "T_long"]].iloc[val_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long", "T_long"]].iloc[train_idx]), np.unique(df[["M_long", "T_long"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long", "T_long"]].iloc[val_idx]), np.unique(df[["M_long", "T_long"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long_dia", "T_long"]].iloc[train_idx]), np.unique(df[["M_long_dia", "T_long"]].iloc[val_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long_dia", "T_long"]].iloc[train_idx]), np.unique(df[["M_long_dia", "T_long"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long_dia", "T_long"]].iloc[val_idx]), np.unique(df[["M_long_dia", "T_long"]].iloc[test_idx]))) == 0
    else:  # last three are split on initiator and terminator
        assert len(np.intersect1d(np.unique(df[["I_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "T_long"]].iloc[val_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "T_long"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "T_long"]].iloc[val_idx]), np.unique(df[["I_long", "T_long"]].iloc[test_idx]))) == 0


### 3D split

In [19]:
split_dimension = 3
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9):
    # import indices
    train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
    val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
    test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()

    # check mutually exclusive
    assert len(np.intersect1d(train_idx, val_idx)) == 0
    assert len(np.intersect1d(train_idx, test_idx)) == 0
    assert len(np.intersect1d(val_idx, test_idx)) == 0

    # check 3D groups are mutually exclusive
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long", "T_long"]].iloc[val_idx]))) == 0
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long", "T_long"]].iloc[test_idx]))) == 0
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long", "T_long"]].iloc[val_idx]), np.unique(df[["I_long", "M_long", "T_long"]].iloc[test_idx]))) == 0
    
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[val_idx]))) == 0
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[test_idx]))) == 0
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[val_idx]), np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[test_idx]))) == 0